Importing the Requried Libraries

In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Importing Datasets

In [43]:
true = pd.read_csv('True.csv')
fake = pd.read_csv('Fake.csv')


<ipython-input-43-314bf9ce5479>:2: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171) have mixed types. Specify dtype option on import or set low_memory=False.
  fake = pd.read_csv('Fake.csv')


In [44]:
# Assigning labels
true["label"] = 1
fake["label"] = 0

In [45]:
# Mergeing datasets
data = pd.concat([true, fake], axis=0)

In [46]:
# Drop unnecessary columns
data = data.drop(["title", "subject", "date"], axis=1)

In [47]:
# Shuffle data
data = data.sample(frac=1).reset_index(drop=True)

Preprocessing the datas

In [48]:
# Preprocessing function
def preprocess_text(text_data):
    preprocessed_text = []
    for sentence in tqdm(text_data):
        sentence = re.sub(r'[^\w\s]', '', sentence)
        preprocessed_text.append(' '.join(
            token.lower() for token in str(sentence).split()
            if token not in stopwords.words('english') or token in ["not", "no"]
        ))
    return preprocessed_text

In [49]:
data['text'] = preprocess_text(data['text'].values)

100%|██████████| 44919/44919 [27:08<00:00, 27.59it/s]


In [50]:
# Feature Engineering
data['word_count'] = data['text'].apply(lambda x: len(x.split()))
data['char_count'] = data['text'].apply(lambda x: len(x))
data['special_char_count'] = data['text'].apply(lambda x: sum(1 for c in x if not c.isalnum()))

In [51]:
# Splitting data
X = data['text']
y = data['label']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [52]:
# Vectorization
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [53]:
# Logistic Regression Model
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [54]:
# Testing & Accuracy
print("Logistic Regression Accuracy:", accuracy_score(y_test, model.predict(x_test)))

Logistic Regression Accuracy: 0.9886910062333036


In [55]:
# Decision Tree Model
tree_model = DecisionTreeClassifier()
tree_model.fit(x_train, y_train)

DecisionTreeClassifier()

In [56]:
# Testing & Accuracy
print("Decision Tree Accuracy:", accuracy_score(y_test, tree_model.predict(x_test)))

Decision Tree Accuracy: 0.9951914514692787


In [57]:
# Manual Testing Function
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)

    # Preprocess text
    pptext= preprocess_text(new_def_test['text'].values)

    # Transform using trained vectorizer
    tf = vectorizer.transform(pptext)

    # Prediction
    yy = model.predict(tf)

    if yy[0] == 0:
        print("Fake News")
    else:
        print("Real News")

In [59]:
# Run manual testing
getinput = input("Enter news text: ")
manual_testing(getinput)

Enter news text: 21st Century Wire says Does Hollywood do anything other than comic book remakes any more? The latest over-budget chimera, Batman V Superman, cost around $410 million to make. You d think they could squeeze a decent film out of that. You d think. He hasn t felt this low since Gigli , said the Huff Post. Movie star Ben Affleck may have just beaten Chris Christie for this year s most vacant stare. Watch: READ MORE HOLLYWOOD NEWS AT: 21st Century Wire Hollywood Files


100%|██████████| 1/1 [00:00<00:00, 108.94it/s]

Fake News
